# Example k3d in discretisedfield

In [1]:
import k3d
import pylab
import matplotlib
import numpy as np
import discretisedfield as df
import matplotlib.pyplot as plt
%matplotlib inline

/Users/sergii.mamedov/Documents/joommf/discretisedfield/discretisedfield/__init__.py:4: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  matplotlib.use("agg")


## Simple sample

Create a 10 x 10 x 10 nm sample. The initial field is directed along the X axis, in a volume bounded by a sphere with a diameter of 5 nm.

In [2]:
size = 5
sphere = 5
mesh = df.Mesh(p1=(-size, -size, -size), p2=(size, size, size), cell=(1, 1, 1))

def val(pos):
    x, y, z = pos
    return y, x, x*y

def norm_fun(pos):
    x, y, z = pos
    if x**2 + y**2 + z**2 < sphere**2:
        return 1
    else:
        return 0

field = df.Field(mesh, value=val, norm=norm_fun)

# Slice vector field

Visualization of a vector field on a selected plane. Using the function `field.mesh.plane (axis = value)` we get the coordinate values of the origin of the vectors and their magnitude. In the process, we filter those vectors whose length is equal to zero.

We obtain the coordinates of the origin of the vectors and their length on the selected plane.

In [3]:
origins_nonzero = []
vectors_nonzero = []
for i in list(field.mesh.plane(z=0.5)):
    vector = field(i)
    value = vector[0]**2 + vector[1]**2 + vector[2]**2
    if value > 0:
        origins_nonzero.append(i)
        vectors_nonzero.append(field(i))

Convert to numpy array

In [4]:
origins_nonzero = np.array(origins_nonzero)
vectors_nonzero = np.array(vectors_nonzero)

Plot

In [5]:
color = 0xff
vector_scale = 0.5

plot = k3d.plot()
plt  = k3d.vectors(origins_nonzero, vector_scale*vectors_nonzero, color=color)
plot += plt
plot.display()

/Users/sergii.mamedov/miniconda3/envs/joommf2/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

# Slice scalor field

Select the field values on the plane

In [6]:
ith = 4
value_slice = field.array[:,:,ith,1]
nz,ny,nx,_  = field.array.shape

Create a list of triples of node numbers

In [7]:
ni,nj = value_slice.shape
indices = []
for i in range(ni-1):
    for j in range(nj-1):
        indices.append((i*nj+j, i*nj+j+1, (i+1)*nj+j))        # lower-left triangle
        indices.append(((i+1)*nj+j+1, i*nj+j+1, (i+1)*nj+j))  # upper-right triangle

Select the mesh coordinate

In [8]:
origins = np.array(list(field.mesh.coordinates))
origins_slice = origins.reshape(nz,ny,nx,3)[:,:,ith,:]

Plot

In [13]:
plot = k3d.plot()
plt_mesh = k3d.mesh(origins_slice, indices, attribute=value_slice,
                    color_map=k3d.basic_color_maps.CoolWarm,
                    color_range=[-1.0, 1.0])
plot += plt_mesh
plot.display()

Output()